# Amazon SageMaker Studio + SSH + IDE Integration

Run this notebook every time when you start a new SageMaker Studio App.

## Environment setup

Run the next cell to get the Kernel Gateway App name, which you will need later to connect to the SageMaker Studio from your local machine. You can find it in the resource metadata file under the key `ResourceName`. Usually, it's the same as the kernel hostname.

In [ ]:
%%sh
hostname
cat /opt/ml/metadata/resource-metadata.json

Next, update pip and install SageMaker SSH Helper library into the kernel app.

In [ ]:
%%sh
pip install -U pip
pip uninstall --root-user-action ignore -y awscli
pip install --root-user-action ignore -q -U sagemaker-ssh-helper
pip freeze | grep sagemaker-ssh-helper

The `configure` command will install the software for remote access, including SSH daemon and VNC server and XFCE4 desktop.

In [ ]:
%%sh
sm-ssh-ide configure

## (Optional) Remote desktop / VNC setup

If you don't plan to run any GUI apps inside SageMaker Studio, skip to the next section or run the cells as is (safe defaults).

Choose your VNC password. It's safe to leave it as is, because your connection will be already secured with SSH and SSM. You need some password, because some VNC clients do not allow passwordless connections.

If you already configured password from another kernel app, this cell will take no effect (the encrypted password file will be shared between apps).

In [ ]:
%%sh
VNC_PASSWORD="123456"  # replace with your password

sm-ssh-ide set-vnc-password "$VNC_PASSWORD"

Specify JetBrains License Server host name in `JB_LICENSE_SERVER_HOST` variable. Leave it as is if you don't use one, e.g. use PyCharm Community or VS Code, or already configured one from another kernel app (the value will be shared between apps).

Please, make sure you've configured the same value on your local machine (see the script `sm-local-ssh-ide`).

In [ ]:
%%sh
JB_LICENSE_SERVER_HOST="jetbrains-license-server.example.com"  # replace with your JetBrains License Server host name

sm-ssh-ide set-jb-license-server "$JB_LICENSE_SERVER_HOST"

## SSM setup

In the next cell you need to put your local user id as the parameter to `init-ssm`. It will help to tag the managed instance with your id as the owner. You can find the id by running locally `aws sts get-caller-identity` command as the value of the returned `UserId`.

If you already configured the id from another kernel app, leave the value as is (it will be loaded from the shared configuration file automatically).

In [ ]:
%%sh
LOCAL_USER_ID="AIDACKCEVSQ6C2EXAMPLE"  # replace with your local UserId

sm-ssh-ide init-ssm "$LOCAL_USER_ID"

## Run SSM Agent & Services

The `start` command will run the background applications for remote access – SSH daemon, VNC server and Jupyter notebook. The `stop` command helps with cleanup from previous runs (see the next section).

If you don't want to run a VNC server and Jupyter notebook, pass the flag `--ssh-only` to the `start` command. Also consider passing the same flag to the `sm-local-ssh-ide` command (you will find it in further instructions).

In [ ]:
%%sh
sm-ssh-ide stop
sm-ssh-ide start
#sm-ssh-ide start --ssh-only

Now execute `ssm-agent` command and keep it running (do not stop the kernel). To validate that the SageMaker Studio app has connected to SSM, go to the Systems Manager Console > Fleet Manager and check that a new managed node appeared in the list and got the "SSM Agent ping status" as Online.

In [ ]:
%%sh
sm-ssh-ide ssm-agent

The above cell will run SSM Agent Daemon infinitely until it stopped.

At this point, complete the instructions for your local machine described in [README.md](README.md#studio).

To stop the Agent, interrupt the kernel.

## Stop SSM Agent & Services

Execute `stop` command to gracefully shut down the SSM Agent Daemon, SSH tunnel, port forwarding and other services.
It's useful when you want to restart the services. If you don't call this command before calling `start` again, you will get a message *Already running on this instance? Call stop first.*

In [ ]:
%%sh
sm-ssh-ide stop